In [1]:
import os
if "x_perceiver" not in os.listdir():
    os.chdir("/home/kh701/pycharm/healnet/")
import torch
from torch import nn
import multiprocessing
import torchvision
import numpy as np
import torchvision.transforms as transforms
import einops
import pandas as pd
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from healnet.models.explainer import Explainer
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

from healnet.utils import Config, flatten_config
from healnet.etl import TCGADataset
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

    
%reload_ext autoreload
%autoreload 2

## Import data

In [2]:
# get dataloaders
config = Config("config/main_gpu.yml").read()
config = flatten_config(config) # TODO - refactor to other 

blca = TCGADataset(
    dataset="blca", 
    config=config, 
    level=2, 
    sources=["omic"]
)

brca = TCGADataset(
    dataset="brca", 
    config=config, 
    level=2, 
    sources=["omic"]
)



Filled 0 missing values with mean
Missing values per feature: 
 Series([], dtype: int64)
Slides available: 436
Omic available: 437
Overlap: 436
Filtering out 1 samples for which there are no omic data available
Dataloader initialised for blca dataset
Dataset: BLCA
Molecular data shape: (436, 2191)
Molecular/Slide match: 436/436
Slide level count: 4
Slide level dimensions: ((79968, 79653), (19992, 19913), (4998, 4978), (2499, 2489))
Slide resize dimensions: w: 1024, h: 1024
Sources selected: ['omic']
Censored share: 0.539
Survival_bin_sizes: {0: 72, 1: 83, 2: 109, 3: 172}
Filled 0 missing values with mean
Missing values per feature: 
 Series([], dtype: int64)
Slides available: 1019
Omic available: 1022
Overlap: 1019
Filtering out 3 samples for which there are no omic data available
Dataloader initialised for brca dataset
Dataset: BRCA
Molecular data shape: (1019, 2922)
Molecular/Slide match: 1019/1019
Slide level count: 3
Slide level dimensions: ((35855, 34985), (8963, 8746), (2240, 218

In [3]:
# get tabular data
blca_loader = DataLoader(
    blca, 
    batch_size=1, 
    shuffle=True, 
    num_workers=multiprocessing.cpu_count()-1
)
[sample], censorship, event_time, y_disc = next(iter(blca_loader))

In [4]:
sample.shape

torch.Size([1, 1, 2183])

## Tabular self-supervised pre-training

To start with, we want to build and encoder-decoder model which trains a cross-attention unit as the encoder, which can later on be deployed in the iterative model. We then want to benchmark the performance with pan-cancer pre-training vs. without pre-training. 

In [5]:
from healnet.models.healnet import Attention, PreNorm

class AttentionEncoder(nn.Module): 
    """
    Simple encoder that uses fourier encoding, pre-norm and cross-attention to encode the input features into a latent array 
    of size (num_latents x latent_dim). Takes in both the input tensors as well as a randomly initialised latent 
    array as the input. 
    """
    def __init__(self, 
                 input_channels: int,
                 latent: torch.Tensor, 
                 input_axis: int = 1, 
                 attn_dropout: float = 0.1,
                 num_heads: int = 4, 
                 num_freq_bands: int=8, 
                 ):    
        super().__init__()
        
        self.input_channels = input_channels
        self.input_axis = input_axis
        self.attn_dropout = attn_dropout
        self.num_heads = num_heads
        
        
        # fourier_channels = (input_axis * ((num_freq_bands * 2) + 1))
        # input_dim = fourier_channels + input_channels
        input_dim = input_channels
                
        latent_dim = latent.shape[-1] # required for PreNorm layer
        # simple single attention unit
        enc = PreNorm(latent_dim, Attention(latent_dim, input_dim, heads=num_heads, dim_head=num_heads, dropout=attn_dropout), context_dim=input_dim)
        
        self.layers = nn.ModuleList([enc])
        
    def forward(self, latent: torch.Tensor, context: torch.Tensor):
        """
        Note: context is the data, x is the latent
        Args:
            latent: 
            context: 

        Returns:

        """
        for layer in self.layers:
            latent = layer(x=latent, context=context)
        return latent


The decoder often needs to be different depending on the modality, so let's implement modality-specific decoders while trying to have a relatively general-purpose encoder that we can plug into the pipeline.

Note that we may change this later down the line. 

In [6]:
class TabularDecoder(nn.Module):
    """
    Decoder suited for tabular data. We use the following: 
    - Skip connections: faster and more stable training
    - Batch normalisation: stabilises the activations and speeds up training
    - Activation: Output layer to map back to output dimensions, corresponding to the original data dims
    Tries to reconstruct the original input given the latent
    """
    def __init__(self, latent_dim: int, num_latents: int, output_dim: int, method: str = "dense"):
        super(TabularDecoder, self).__init__()
        assert method in ["dense", "conv"], "Decoder type not recognised"
        # check that latent_dim is divisible by 4
        assert num_latents % 4 == 0, "Latent dim must be a multiple of 4"
        layers = []
        
        if method == "dense": 
            
            # flatten latent array (batch, num_latents, latent_dim) -> (batch, num_latents * latent_dim)
            layers.extend([nn.Flatten()]) 
            out_dims = [1024, 512, 256] # may refactor as hyperparameter later
            
            in_dim = latent_dim * num_latents
            for idx, out_dim in enumerate(out_dims):
                
                layers.extend([
                    nn.Linear(in_features=in_dim, out_features=out_dim), 
                    nn.LeakyReLU(), 
                    nn.InstanceNorm1d(out_dim, track_running_stats=False), 
                    nn.Dropout(0.5)
                ])
                
                in_dim = out_dim # update for next layer
            
            # final layer to reconstruct output
            layers.append(nn.Linear(in_dim, output_dim))
        
        elif method == "conv": 
            print(latent_dim, num_latents)
            layers.extend([
                nn.ConvTranspose1d(num_latents, out_channels=int(num_latents/2), kernel_size=4, stride=2, padding=1), 
                nn.BatchNorm1d(int(num_latents/2)),
                nn.LeakyReLU(negative_slope=0.1),
                
                nn.ConvTranspose1d(int(num_latents/2), out_channels=int(num_latents/4), kernel_size=4, stride=2, padding=1),
                nn.BatchNorm1d(int(num_latents/4)),
                nn.LeakyReLU(negative_slope=0.1),
                
                # If you added any other ConvTranspose layers, ensure the channel sizes match correctly for those as well.
                
                nn.Conv1d(int(num_latents/4), out_channels=1, kernel_size=1, stride=1, padding=0)
            ])
        
        self.decode = nn.Sequential(*layers)
        print(self.decode)
        
    def forward(self, latent: torch.Tensor):
        return self.decode(latent)
    
    
        
    

Finally, putting it all together in the encoder-decoder model


In [7]:
from typing import *

class TabPretrainer(nn.Module): 
    """
    Encoder-decoder model for pre-training tabular data.
    # TODO - refactor abstract base class for initialisations 
    """
    def __init__(self,
                 sample: torch.Tensor,
                 # input_channels: int,
                 latent_shape: List[int],
                 input_axis: int = 1,
                 attn_dropout: float = 0.1,
                 num_heads: int = 4,
                 num_freq_bands: int=8,
                 ):
        super().__init__()
        self.input_channels = sample.shape[-1]
        self.input_axis = input_axis
        self.num_latents, self.latent_dim = latent_shape
        # self.latent_dim, self.num_latents = latent_shape
        self.attn_dropout = attn_dropout
        self.num_heads = num_heads
        self.num_freq_bands = num_freq_bands
        
        
        # randomly initialise latent
        self.latent = nn.Parameter(torch.randn(self.num_latents, self.latent_dim))
        
        # encoder
        self.encoder = AttentionEncoder(
            input_channels=self.input_channels, 
            latent=self.latent, 
            input_axis=self.input_axis, 
            attn_dropout=attn_dropout, 
            num_heads=num_heads, 
            num_freq_bands=num_freq_bands
        )
        
        # decoder
        self.decoder = TabularDecoder(
            latent_dim=self.latent_dim, 
            num_latents=self.num_latents,
            output_dim=self.input_channels,
            # method="conv"
            method="dense" # using simple encoder to force good representation
        )
        
    def forward(self, x: torch.Tensor):
        # get batch dim
        b = x.shape[0]
        
        # expand latent to batch size
        if len(self.latent.shape) == 2:
            self.latent = nn.Parameter(einops.repeat(self.latent, "n d -> b n d", b=b))
        
        # encode
        # self.latent.data = self.encoder(latent=self.latent, context=x).data + self.latent.data
        # works much better with skip connections
        self.latent.data = self.encoder(latent=self.latent, context=x).data 
        # decode, reconstructed x
        rec_x = self.decoder(self.latent)
        return rec_x
    
    def get_latent(self):
        return self.latent

Next, we need to think about tabular loss functions. Here, we can explore both reconstruction losses and contrastive losses. 

In [8]:
class TabularLoss(nn.Module):
    """
    Reconstruction loss functions for tabular data. We use two types which are commonly used with continuous data: 
    - Mean squared error
    - Constrastive loss, measured as cosine distance between the original and reconstructed data
    We seek to minimise both objectives.
    """
    def __init__(self,
                 method: str = "mse",
                 reduction: str = "mean",
                 ):
        super().__init__()
        assert method in ["mse", "contrastive"], "Loss type not recognised"
        self.loss_type = method
        self.reduction = reduction
        
        if method == "mse":
            self.loss = nn.MSELoss(reduction=reduction)
        elif method == "contrastive":
            self.loss = nn.CosineEmbeddingLoss(reduction=reduction)
            
    def __call__(self, **kwargs):
        return self.loss(**kwargs)
    

Finally, we write a pre-training loop that we can use for pre-training across cancer sites. 

In [9]:
from tqdm import tqdm

torch.set_printoptions(sci_mode=False)


def pretrain_loop(
        data: TCGADataset,
        batch_size: int,
        epochs: int = 10, 
    ):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    loader = DataLoader(
        data, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=multiprocessing.cpu_count()-1
    )
    [omic_sample], _, _, _ = next(iter(loader))   
    
    
    model = TabPretrainer(
        sample = omic_sample,
        # input_channels=omic_sample.shape[-1], 
        input_axis=1, 
        latent_shape=[256, 32], 
        attn_dropout=0.1, 
        num_heads=1,
        num_freq_bands=8
    )
    model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    
    loss_method = "mse"
    loss_fn = TabularLoss(method=loss_method)
    
    for epoch in tqdm(range(epochs)):
        for idx, batch in enumerate(loader):
            [omic], censorship, event_time, y_disc = batch
            omic = omic.to(device)
            rec_omic = model(omic)
            # print(rec_omic.shape)
            # print(omic.shape)
            if loss_method == "contrastive":
                # need to pass in larges for contrastive loss
                # using torch.ones to ensure that omic and rec_omic are learned as similar representations
                # note that this is a slight repurposing of the contrastive loss function
                # with this, the loss is just 1-cos(omic, rec_omic)
                loss = loss_fn(input1=omic, input2=rec_omic, target=torch.ones(omic.shape[0]))
            elif loss_method == "mse": 
                loss = loss_fn(input=omic, target=rec_omic)
            # loss = loss_fn(omic, rec_omic)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            # print every 10th batch
            if idx % 100 == 0:
                pass
                # print(loss)
                # print(omic)
                # print(rec_omic)
        # print epoch-level stats
        print(f"Epoch {epoch+1} loss: {loss}")
        # final reconstruction
        # print error vector
        # print((omic - rec_omic).abs())
        # print(omic)
        # print(rec_omic)
    return model
        
            
    
tab_encoder = pretrain_loop(
                data=blca,
                batch_size=1,
                epochs=5
            ) 
    

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=8192, out_features=1024, bias=True)
  (2): LeakyReLU(negative_slope=0.01)
  (3): InstanceNorm1d(1024, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (4): Dropout(p=0.5, inplace=False)
  (5): Linear(in_features=1024, out_features=512, bias=True)
  (6): LeakyReLU(negative_slope=0.01)
  (7): InstanceNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (8): Dropout(p=0.5, inplace=False)
  (9): Linear(in_features=512, out_features=256, bias=True)
  (10): LeakyReLU(negative_slope=0.01)
  (11): InstanceNorm1d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  (12): Dropout(p=0.5, inplace=False)
  (13): Linear(in_features=256, out_features=2183, bias=True)
)


/home/kh701/.conda/envs/cognition/lib/python3.9/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
  0%|          | 0/5 [00:00<?, ?it/s]

tensor(-0.9595, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.9677, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0048, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.0144, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0217, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.0504, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.1447, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.3338, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.1060, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.2454, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda

/home/kh701/.conda/envs/cognition/lib/python3.9/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([1, 2183])) that is different to the input size (torch.Size([1, 1, 2183])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(0.0991, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.2298, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.2218, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.5082, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.1661, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.1874, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-0.1076, device='cuda:0', grad_fn=<MinBackward1>) tensor(-0.0470, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-0.0164, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.2966, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cu

...

Epoch 3 loss: 1.0081923007965088
tensor(0.0361, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.0520, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-0.0700, device='cuda:0', grad_fn=<MinBackward1>) tensor(-0.0495, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-0.0867, device='cuda:0', grad_fn=<MinBackward1>) tensor(-0.0408, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(-0.0365, device='cuda:0', grad_fn=<MinBackward1>) tensor(-0.0181, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.1016, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.2212, device='cuda:0', grad_fn=<

 80%|████████  | 4/5 [00:15<00:03,  3.86s/it]

Epoch 4 loss: 1.5907981395721436
tensor(0.1286, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.2990, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.1589, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.3740, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0626, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.1272, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0380, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.0849, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
tensor(0.0583, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.1031, device='cuda:0', grad_fn=<MaxBac

100%|██████████| 5/5 [00:19<00:00,  3.91s/it]

Epoch 5 loss: 1.6420265436172485


In [25]:
a = torch.rand(4, 17, 1)
a.softmax(dim=-2)

tensor([[[0.0522],
         [0.0441],
         [0.0379],
         [0.0382],
         [0.0497],
         [0.0586],
         [0.0902],
         [0.0659],
         [0.0859],
         [0.0689],
         [0.0371],
         [0.0592],
         [0.0783],
         [0.0750],
         [0.0710],
         [0.0515],
         [0.0366]],

        [[0.0821],
         [0.0477],
         [0.0511],
         [0.0943],
         [0.0655],
         [0.0673],
         [0.0832],
         [0.0530],
         [0.0430],
         [0.0377],
         [0.0775],
         [0.0365],
         [0.0352],
         [0.0386],
         [0.0623],
         [0.0493],
         [0.0758]],

        [[0.0421],
         [0.0829],
         [0.0759],
         [0.0693],
         [0.0419],
         [0.0415],
         [0.0554],
         [0.0407],
         [0.0377],
         [0.0425],
         [0.0777],
         [0.0585],
         [0.0862],
         [0.0383],
         [0.0825],
         [0.0570],
         [0.0699]],

        [[0.0632],
      

In [10]:
import matplotlib.pyplot as plt
import seaborn as sns
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# pass through sample
[sample], _, _,_ = next(iter(blca_loader))
sample = sample.to(device)
tab_encoder.eval()
rec_sample = tab_encoder(sample)

# access layers in model
attn_weights = tab_encoder.encoder.layers[0].fn.attn_weights
# detach
attn_weights = attn_weights.squeeze().detach().cpu().numpy()
print(attn_weights.shape)

# # look at attention distribution
# def plot_attn_weights(attn_weights: np.ndarray, batch_idx: int = 0):
#     sns.set_theme()
#     attn_weights = attn_weights[batch_idx]
#     fig, ax = plt.subplots(figsize=(10, 10))
#     sns.histplot(attn_weights, ax=ax)
#     plt.show()
# 
# plot_attn_weights(attn_weights)

tensor(0.0799, device='cuda:0', grad_fn=<MinBackward1>) tensor(0.1857, device='cuda:0', grad_fn=<MaxBackward1>)
tensor(1., device='cuda:0', grad_fn=<MinBackward1>) tensor(1., device='cuda:0', grad_fn=<MaxBackward1>)
(256,)


In [11]:
attn_weights

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.